In [5]:
import statsapi
import requests
import json
BASE_URL = "https://statsapi.mlb.com/api"

from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

sched = statsapi.schedule(date=today)
print(sched)

games = []


for game in sched:
    games.append({"time" : game['game_datetime'], "away" : {'id': game['away_id'], 'sp' : game['away_probable_pitcher']}, "home": {'id': game['home_id'], 'sp' : game['away_probable_pitcher']}})
# # for game in sched:
games
    
def check_div_index(div_id, league_id):
    div_index = -1
    if league_id == 103:
        # al team
        if div_id == 201:
            # al east
            div_index = 0
        elif div_id == 202:
            # al central
            div_index = 1
        elif div_id == 200:
            # al west
            div_index = 2
    elif league_id == 104:
        # nl team
        if div_id == 204:
            # nl east
            div_index = 0
        elif div_id == 205:
            # nl central
            div_index = 1
        elif div_id == 203:
            # nl west
            div_index = 2
    return div_index




[{'game_id': 746866, 'game_datetime': '2024-06-16T17:05:00Z', 'game_date': '2024-06-16', 'game_type': 'R', 'status': 'Final', 'away_name': 'St. Louis Cardinals', 'home_name': 'Chicago Cubs', 'away_id': 138, 'home_id': 112, 'doubleheader': 'N', 'game_num': 1, 'home_probable_pitcher': 'Jameson Taillon', 'away_probable_pitcher': 'Miles Mikolas', 'home_pitcher_note': '', 'away_pitcher_note': '', 'away_score': 2, 'home_score': 1, 'current_inning': 9, 'inning_state': 'Bottom', 'venue_id': 17, 'venue_name': 'Wrigley Field', 'national_broadcasts': ['Roku'], 'series_status': 'STL wins 2-1', 'winning_team': 'St. Louis Cardinals', 'losing_team': 'Chicago Cubs', 'winning_pitcher': 'Miles Mikolas', 'losing_pitcher': 'Jameson Taillon', 'save_pitcher': 'Ryan Helsley', 'summary': '2024-06-16 - St. Louis Cardinals (2) @ Chicago Cubs (1) (Final)'}, {'game_id': 747023, 'game_datetime': '2024-06-16T17:35:00Z', 'game_date': '2024-06-16', 'game_type': 'R', 'status': 'Final', 'away_name': 'Philadelphia Phill

In [6]:
# print(games)
# print(games)

for game in games:
    # print(game)
    try:
        response_away = requests.get(f"{BASE_URL}/v1/teams/{game['away']['id']}")
        game['away']['div_id'] = response_away.json()['teams'][0]['division']['id']
        game['away']['league_id'] = response_away.json()['teams'][0]['league']['id']
        response_home = requests.get(f"{BASE_URL}/v1/teams/{game['home']['id']}")
        game['home']['div_id'] = response_home.json()['teams'][0]['division']['id']
        game['home']['league_id'] = response_home.json()['teams'][0]['league']['id']
        print(game)
    except requests.exceptions.RequestException as e:
        print(e)

for game in games:
    away_div_id = game['away']['div_id']
    home_div_id = game['home']['div_id']
    away_league_id = game['away']['league_id']
    home_league_id = game['home']['league_id']
    game['away']['div_index'] = check_div_index(away_div_id, away_league_id)
    game['home']['div_index'] = check_div_index(home_div_id, home_league_id)
    



{'time': '2024-06-16T17:05:00Z', 'away': {'id': 138, 'sp': 'Miles Mikolas', 'div_id': 205, 'league_id': 104}, 'home': {'id': 112, 'sp': 'Miles Mikolas', 'div_id': 205, 'league_id': 104}}
{'time': '2024-06-16T17:35:00Z', 'away': {'id': 143, 'sp': 'Zack Wheeler', 'div_id': 204, 'league_id': 104}, 'home': {'id': 110, 'sp': 'Zack Wheeler', 'div_id': 201, 'league_id': 103}}
{'time': '2024-06-16T17:35:00Z', 'away': {'id': 139, 'sp': 'Zach Eflin', 'div_id': 201, 'league_id': 103}, 'home': {'id': 144, 'sp': 'Zach Eflin', 'div_id': 204, 'league_id': 104}}
{'time': '2024-06-16T17:35:00Z', 'away': {'id': 146, 'sp': 'Jesús Luzardo', 'div_id': 204, 'league_id': 104}, 'home': {'id': 120, 'sp': 'Jesús Luzardo', 'div_id': 204, 'league_id': 104}}
{'time': '2024-06-16T17:37:00Z', 'away': {'id': 114, 'sp': 'Ben Lively', 'div_id': 202, 'league_id': 103}, 'home': {'id': 141, 'sp': 'Ben Lively', 'div_id': 201, 'league_id': 103}}
{'time': '2024-06-16T17:40:00Z', 'away': {'id': 135, 'sp': 'Dylan Cease', 'div_

In [ ]:

for game in games:
    away_div_index = game['away']['div_index']
    away_league_id = game['away']['league_id']
    away_team_id = game['away']['id']
    home_div_index = game['home']['div_index']
    home_league_id = game['home']['league_id']
    home_team_id = game['home']['id']
    home_response = requests.get(f"https://statsapi.mlb.com/api/v1/standings?leagueId={home_league_id}&season=2024&standingsTypes=regularSeason")
    away_response = requests.get(f"https://statsapi.mlb.com/api/v1/standings?leagueId={away_league_id}&season=2024&standingsTypes=regularSeason")
    if away_response.status_code == 200:
        away_json = away_response.json()
        away_division_data = away_json['records'][away_div_index]['teamRecords']
        for team in away_division_data:
            if team['team']['id'] == away_team_id:
                game['away']['team_data'] = team
    if home_response.status_code == 200:
        home_json = home_response.json()
        home_division_data = home_json['records'][home_div_index]['teamRecords']
        for team in home_division_data:
            if team['team']['id'] == home_team_id:
                game['home']['team_data'] = team






KeyError: 'div_index'